Ring Polymer Melt
===========

insert description

### Concepts Used
- blergh

### Tools Used
- blergh

References
-----------
1. Curro, J.G.; Schweizer K.S.; Theory of Polymer Melts: An Integral Equation Approach
   Macromolecules, 1987, 20, 1928-1934, doi: 10.1021/ma00174a040
 
  

Global Imports and Setup
----------------------------

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh')

%load_ext autoreload
%autoreload 2

In [ ]:
%opts Curve Scatter [width=500,height=400] Layout [shared_axes=False] Scatter (size=10,alpha=0.5)
%opts Curve Scatter [fontsize={'xlabel':14,'xlabel':14,'ylabel':14,'ticks':12}]
%opts Overlay [legend_position='bottom_left']
%opts Layout [shared_axes=False]


colors = {}
colors[2000] = 'blue'
colors[500] = 'red'
colors[100] = 'green'

ls = {}
ls[2000] = 'solid'
ls[500] = 'dashed'
ls[100] = 'dotted'

markers = {}
markers[2000] = 'o'
markers[500] = '^'
markers[100] = 'd'

Demo
-----

First we'll load in the data extracted from the literature that we'll be comparing against. 

In [ ]:
gr_compare = []
for density in [2000,500,100]:
    fname = './data/RingMelt-Gr-N{}.csv'.format(density)
    x,y = np.loadtxt(fname,delimiter=',').T
    gr_compare.append([density,x,y])


Now we do the PRISM calculation...

In [ ]:
import typyPRISM
from typyPRISM.calculate.prism.pair_correlation import pair_correlation

sys = typyPRISM.System(['polymer'],kT=1.0)
sys.domain = typyPRISM.Domain(dr=0.05,length=4096)
sys.diameter['polymer'] = 1.0
sys.density['polymer'] = 0.9
sys.closure['polymer','polymer'] = typyPRISM.closure.PercusYevick()
sys.potential['polymer','polymer'] = typyPRISM.potential.HardSphere(sigma=1.0)

gr_results = []
guess = np.zeros_like(sys.domain.r)
for length in [2000,500,100]:
    print('==> Solving for gaussian ring of length {}'.format(length))
    sys.omega['polymer','polymer'] = typyPRISM.omega.GaussianRing(sigma=1.0,length=length)
    PRISM = sys.createPRISM()
    result = PRISM.solve(guess)
    guess = np.copy(PRISM.x)
    
    x = sys.domain.r
    y = pair_correlation(PRISM)['polymer','polymer']
    gr_results.append(['ring',length,x,y])
    
    

Here we reproduce Figure 1 of Reference 1.

In [ ]:
%%opts Overlay [legend_position='bottom_right']
from math import sqrt

extents = (0,0,13,1.0)

gr_plots = []
for name,N,x,y in gr_results:
    label = 'N={} (typyPRISM)'.format(N)
    style = {'line_dash':ls[N],'color':colors[N]}
    c1 = hv.Curve((x,y),label=label,extents=extents)(style=style)
    gr_plots.append(c1)
    
for N,x,y in gr_compare:
    label = 'N={} (Ref 1)'.format(N)
    style = {'marker':markers[N],'color':colors[N]}
    c1 = hv.Scatter((x,y),label=label,extents=extents)(style=style)
    gr_plots.append(c1)
    
    
hv.Overlay(gr_plots).redim.label(x='r',y='g(r)')